Import standard python data science libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Install arcGIS and import libraries. Once installed, these can be commented out

In [3]:
#!pip install geopy

In [4]:
#!pip install arcgis

In [5]:
from arcgis.gis import GIS
from arcgis.geocoding import geocode, reverse_geocode
from arcgis.geometry import Point

Load data on each hurricane. Includes a) Core dataframe with tweet information and b) 4 additional dataframes containing extracted address features:

    highways and cities/counties
    other roads
    mile markers
    highway exits


In [177]:
harvey_df = pd.read_csv("../Data/harvey_clean.csv").fillna("")
harvey_highways = pd.read_csv("../Data/harvey_new_values.csv").fillna("")
harvey_other_roads = pd.read_csv("../Data/harvey_other_roads.csv").fillna("")
harvey_mm = pd.read_csv("../Data/harvey_mm.csv").fillna("")
harvey_exits = pd.read_csv("../Data/harvey_exits.csv").fillna("")

In [178]:
harvey = pd.concat([harvey_df,
           harvey_highways,
           harvey_other_roads,
           harvey_mm,
           harvey_exits], axis=1).drop(columns = "Unnamed: 0")

In [8]:
florence_df = pd.read_csv("../Data/florence_clean.csv").fillna("")
florence_highways = pd.read_csv("../Data/florence_new_values.csv").fillna("")
florence_other_roads = pd.read_csv("../Data/florence_other_roads.csv").fillna("")
florence_mm = pd.read_csv("../Data/florence_mm.csv").fillna("")
florence_exits = pd.read_csv("../Data/florence_exits.csv").fillna("")

In [9]:
florence = pd.concat([florence_df,
           florence_highways,
           florence_other_roads,
           florence_mm,
           florence_exits], axis=1).drop(columns = "Unnamed: 0")

In [10]:
michael_df = pd.read_csv("../Data/michael_clean.csv").fillna("")
michael_highways = pd.read_csv("../Data/michael_new_values.csv").fillna("")
michael_other_roads = pd.read_csv("../Data/michael_other_roads.csv").fillna("")
michael_mm = pd.read_csv("../Data/michael_mm.csv").fillna("")
michael_exits = pd.read_csv("../Data/michael_exits.csv").fillna("")

In [11]:
michael = pd.concat([michael_df,
           michael_highways,
           michael_other_roads,
           michael_mm,
           michael_exits], axis=1).drop(columns = "Unnamed: 0")

In [12]:
michael["county_string"] = michael["county_string"].fillna("")

In [13]:
michael = michael[michael["state"] != "Georgia"]

In [14]:
def string_from_list(values_list):
    values_string = ""
    if(len(values_list) > 0):
        for i in range(len(values_list)):  
            values_string = str(values_string) + str(values_list[i]) + ","
    else:
        values_string = "None "
    return values_string[:-1]

### Create list of potential addresses

First function creates list of strings that represent all potential combinations of 1) city, state or  2) county, state for each tweet in the database. If no city or county info is available, address is stored as just the state

The second function below incorporates specific streets into the list of addresses

In [15]:
def create_city_county_addresses(df):
    address_lists = []
    for rows in df.index:
        address_list = []
        state = df.loc[rows, "state"]
        if(df.loc[rows, "county_string"] != "None"):
            county_list = list(set(df.loc[rows, "county_string"].split(",")))
            for county in county_list:
                address_list.append(county + ", " + state + ", USA")
        if((df.loc[rows, "city_string"] != "None") & (df.loc[rows, "city_string"] != "")):
            city_list = list(set(df.loc[rows, "city_string"].split(",")))
            for city in city_list:
                address_list.append(city + ", " + state + ", USA")
        address_list.append(state + ", USA")
        address_lists.append(address_list)
    return address_lists

In [100]:
def create_full_address_lists(df):
# Call city/county function to create list of possible addresses in city, state or county, state format
# Declare new empty list of lists to populate with fill addresses
    address_suffix = create_city_county_addresses(df)
    full_address_lists = []
    for rows in df.index:
# For each row of data, create and populate list of potential full addresses
        full_address_list = []
# Unpack all features in dataframe into lists as they are currently stored as strings
# Here, we assume that if multiple highways are present, then the road restriction occurred on the first highway mentioned
        highway_list = list(set(df.loc[rows,"highway_string"].split(",")))
        first_highway = highway_list[0]
        exit_list = list(set(df.loc[rows,"exit_list"].split(",")))
        mm_list = list(set(df.loc[rows,"mm_list"].split(",")))
        other_road_list = list(set(df.loc[rows,"Other_Road_List"].split(",")))
#
        if df.loc[rows,"has_highway"] == 1:
            count = 0
            if(len(address_suffix) > 1):
                for suffix in address_suffix[rows]:            
                    full_address_list.append(first_highway + ", " + suffix)            
            if(len(highway_list) > 1):
                count += 1                
                for i in range(1,len(highway_list)):
                    for suffix in address_suffix[rows]:
                        full_address_list.append(first_highway + " & " + highway_list[i] + ", " + suffix)
            if(exit_list[0] != "None"):
                count += 1                
                for i in range(len(exit_list)):
                    for suffix in address_suffix[rows]:
                        full_address_list.append(first_highway + ", " + exit_list[i] + ", " + suffix)
            if(mm_list[0] != "None"):
                count += 1
                for i in range(len(mm_list)):
                    for suffix in address_suffix[rows]:
                        full_address_list.append(first_highway + ", " + mm_list[i] + ", " + suffix)
            if((other_road_list[0] != "None") & (count == 0)):
                for i in range(len(other_road_list)):
                    for suffix in address_suffix[rows]:
                        full_address_list.append(first_highway + " & " + other_road_list[i] + ", " + suffix)
        full_address_lists.append(full_address_list)
    return full_address_lists

In [180]:
harvey_addresses = create_full_address_lists(harvey)

In [102]:
michael_addresses = create_full_address_lists(michael)

In [103]:
florence_addresses = create_full_address_lists(florence)

In [339]:
gis = GIS("https://www.arcgis.com",
          "dsi_project_5",
          "jbb_project_3",)

In [105]:
def get_coordinates(df, address_list):
    x_list = []
    y_list = []
    match_score_list = []
    matched_address_list = []
    for rows in df.index:
        x = []
        y = []
        match_score = []
        matched_address = []
        if ((rows > 50) & (rows < 100)):
            for address in address_list[rows]:
                try:
                    if(address.find("exit") != -1):
                        try:
                            float(address[address.find("exit") + 5])
                            geocode_req = geocode(address = address, category = "Highway Exit")
                            x.append(geocode_req[0]["location"]["x"])
                            y.append(geocode_req[0]["location"]["y"])
                            match_score.append(geocode_req[0]["score"])
                            matched_address.append(address)
                        except:
                            try:
                                geocode_req = geocode(address)
                                x.append(geocode_req[0]["location"]["x"])
                                y.append(geocode_req[0]["location"]["y"])
                                match_score.append(geocode_req[0]["score"])
                                matched_address.append(address)
                            except:
                                pass
                    else:
                        try:
                            geocode_req = geocode(address)
                            x.append(geocode_req[0]["location"]["x"])
                            y.append(geocode_req[0]["location"]["y"])
                            match_score.append(geocode_req[0]["score"])
                            matched_address.append(address)
                        except:
                            pass
                except:
                    pass
            if(len(x) == 0):
                x = ["None"]
                y = ["None"]
                match_score = ["None"]
                matched_address = ["None"]
            x_string = string_from_list(x)
            y_string = string_from_list(y)
            match_score_string = string_from_list(match_score)
            matched_address_string = string_from_list(matched_address)  
            x_list.append(x_string) 
            y_list.append(y_string)
            match_score_list.append(match_score_string) 
            matched_address_list.append(matched_address_string)
    data = pd.DataFrame()
    data["x"] = x_list
    data["y"] = y_list
    data["match_scores"] = match_score_list
    data["matched_addresses"] = matched_address_list
    return data

In [106]:
michael_coords = get_coordinates(michael, michael_addresses)

In [107]:
michael_coords

,x,y,match_scores,matched_addresses
0,"-83.15699107853857,-82.25938808381903","30.37338952227392,26.894883532315905","100,87.66","I-10, Florida, USA,I-10, mile marker 185, Flor..."
1,"-83.15699107853857,-82.25987841618097","30.37338952227392,26.894937456575605","100,87.66","I-10, Florida, USA,I-10, mile marker 182, Flor..."
2,"-83.15699107853857,-82.26062991618096","30.37338952227392,26.894936482110833","100,87.66","I-10, Florida, USA,I-10, mile marker 176, Flor..."
3,"-83.15699107853857,-82.2633615","30.37338952227392,26.894892532315897","100,87.66","I-10, Florida, USA,I-10, mile marker 153, Flor..."
4,"-74.21516895680448,-84.867291,-74.215168956804...","42.88395503083169,29.743254000000007,42.883955...","100,95.43,81.9,88.6,90.08,87.27,95.68","SR 30, Florida, USA,SR 30 & 4 rig heights, Flo..."
5,"-74.21516895680448,-84.8696985,-74.21516895680...","42.88395503083169,29.741787000000002,42.883955...","100,95.43,81.9,95.68,88.6,90.08,87.27","SR 30, Florida, USA,SR 30 & 2 rig heights, Flo..."
6,"-74.21516895680448,-84.8685735,-84.8685735,-74...","42.88395503083169,29.742525000000004,29.742525...","100,95.68,95.43,81.9,88.6,90.08,87.27","SR 30, Florida, USA,SR 30 & 3 right lane, Flor..."
7,"-83.15699107853857,-82.26292861852951","30.37338952227392,26.898851607565252","100,87.24","I-10, Florida, USA,I-10, mile marker 95, Flori..."
8,"-83.15699107853857,-84.69513999999998","30.37338952227392,30.577000000000055","100,95.54","I-10, Florida, USA,I-10, exit 174, Florida, USA"
9,"-74.21516895680448,-82.47819999999996,-86.6393...","42.88395503083169,27.972300000000075,30.407643...","100,85.48,83.3,81.9,91.72,88.6,90.08,87.27,98....","SR 30, Florida, USA,SR 30 & pkwy rig heights, ..."


In [ ]:
michael_coords.to_csv("../Data/michael_lat_long.csv")

In [ ]:
florence_coords = get_coordinates(florence, florence_addresses)

In [ ]:
florence_coords.to_csv("../Data/florence_lat_long.csv")

In [ ]:
harvey_coords = get_coordinates(harvey, harvey_addresses)

In [ ]:
harvey_coords.to_csv("../Data/harvey_lat_long.csv")

In [ ]:
gis.map(location = (42.331890000000044,-71.32166999999998), zoomlevel = 10)

In [120]:
michael_coords

,x,y,match_scores,matched_addresses
0,"-83.15699107853857,-82.25938808381903","30.37338952227392,26.894883532315905","100,87.66","I-10, Florida, USA,I-10, mile marker 185, Flor..."
1,"-83.15699107853857,-82.25987841618097","30.37338952227392,26.894937456575605","100,87.66","I-10, Florida, USA,I-10, mile marker 182, Flor..."
2,"-83.15699107853857,-82.26062991618096","30.37338952227392,26.894936482110833","100,87.66","I-10, Florida, USA,I-10, mile marker 176, Flor..."
3,"-83.15699107853857,-82.2633615","30.37338952227392,26.894892532315897","100,87.66","I-10, Florida, USA,I-10, mile marker 153, Flor..."
4,"-74.21516895680448,-84.867291,-74.215168956804...","42.88395503083169,29.743254000000007,42.883955...","100,95.43,81.9,88.6,90.08,87.27,95.68","SR 30, Florida, USA,SR 30 & 4 rig heights, Flo..."
5,"-74.21516895680448,-84.8696985,-74.21516895680...","42.88395503083169,29.741787000000002,42.883955...","100,95.43,81.9,95.68,88.6,90.08,87.27","SR 30, Florida, USA,SR 30 & 2 rig heights, Flo..."
6,"-74.21516895680448,-84.8685735,-84.8685735,-74...","42.88395503083169,29.742525000000004,29.742525...","100,95.68,95.43,81.9,88.6,90.08,87.27","SR 30, Florida, USA,SR 30 & 3 right lane, Flor..."
7,"-83.15699107853857,-82.26292861852951","30.37338952227392,26.898851607565252","100,87.24","I-10, Florida, USA,I-10, mile marker 95, Flori..."
8,"-83.15699107853857,-84.69513999999998","30.37338952227392,30.577000000000055","100,95.54","I-10, Florida, USA,I-10, exit 174, Florida, USA"
9,"-74.21516895680448,-82.47819999999996,-86.6393...","42.88395503083169,27.972300000000075,30.407643...","100,85.48,83.3,81.9,91.72,88.6,90.08,87.27,98....","SR 30, Florida, USA,SR 30 & pkwy rig heights, ..."


In [191]:
florida_master_roads = pd.read_csv("../Data/Florida_master_roads.csv").drop(columns = "Unnamed: 0")
ncarolina_master_roads = pd.read_csv("../Data/North Carolina_master_roads.csv").drop(columns = "Unnamed: 0")
scarolina_master_roads = pd.read_csv("../Data/South Carolina_master_roads.csv").drop(columns = "Unnamed: 0")
texas_master_roads = pd.read_csv("../Data/Texas_master_roads.csv").drop(columns = "Unnamed: 0")
virginia_master_roads = pd.read_csv("../Data/Virginia_master_roads.csv").drop(columns = "Unnamed: 0")

In [193]:
master_road_dict = {"Florida" : florida_master_roads,
                    "North Carolina" : ncarolina_master_roads,
                    "South Carolina" : scarolina_master_roads,
                    "Texas" : texas_master_roads,
                    "Virginia" : virginia_master_roads}

In [351]:
def best_location(df):
#Create lists to store best match from each tweet
    best_x = []
    best_y = []
    best_score = []
    best_address = []
        
    for rows in df.index:
# recreate lists of relevant geocoordinate values in each row of passed dataframe
# (currently stored as comma separated strings)
        x_list = df.loc[rows,"x"].split(",")
        y_list = df.loc[rows,"y"].split(",")
        match_scores = df.loc[rows,"match_scores"].split(",")
        matched_address_list = df.loc[rows,"matched_addresses"].split("USA")
# remove trailing commas and spaces from end of addresses
        matched_address_list.pop(len(matched_address_list)-1)
        clean_address = [x[:-2] for x in matched_address_list]
# Create dataframe of these lists such that they can be sorted together
        scores_df = pd.DataFrame()
        scores_df["x_list"] = x_list
        scores_df["y_list"] = y_list
        scores_df["match_scores"] = match_scores
        scores_df["clean_address"] = clean_address
# Sort based on match score to enable iterating down list and picking best score that also meets other criteria
        scores_df = scores_df.sort_values(by = "match_scores", ascending = False).reset_index(drop=True)
#REPLACE MICHAEL !!!!!!!!!!!!!!!!!!!!!!!!!!!!!        
# Set up variables to check if first highway is used elsewhere in the address as this would be redundant
# and can artifiially inflate the match score.
# Includes 1) finding first highway, 2) generating list of abbreviations for that highway,
# 3) removing first highway from addrees
# and 4) searching over remaining piece of address to see if highway is found again.
# If the highway is found, then the best_match remains false and the loop iterates to the address with the 
# next highest match score until a match is found, the loop exits, and the best metrics are added to appropriate lists
#Note that although a scoreof 100 = perfect match, we exclude this as the only way to get a perfect match is
# to have a highway with no intersection or no matching  
        highway_list = list(set(michael.loc[rows+51,"highway_string"].split(",")))
        current_state = michael.loc[rows+51, "state"]
        first_highway = highway_list[0]
        highway_abbrevs = (master_road_dict[current_state]
                           [master_road_dict[current_state]["road"] == first_highway]
                           ["abbrevs"].str.split(",").to_list()[0])
        scores_df["replace_index"] = scores_df["clean_address"].map(lambda x: x.find(first_highway) + len(first_highway))
        scores_df["highway_removed"] = [x[i:] for x, i in zip(scores_df["clean_address"].to_list(), 
                                                              scores_df["replace_index"].to_list())]
        df_length = scores_df.shape[0]
        index = 0
        best_index = 0
        
        for i in range(scores_df.shape[0]):
            x_temp = scores_df.loc[index, "x_list"]
            y_temp = scores_df.loc[index, "y_list"]
            address = scores_df.loc[index, "clean_address"]
            score_temp = scores_df.loc[index, "match_scores"]
            count = 0
            for abbrev in highway_abbrevs:
                if (scores_df.loc[i, "highway_removed"].find(abbrev) != -1): 
                    best_index = i + 1
        best_x.append(scores_df.loc[best_index, "x_list"])
        best_y.append(scores_df.loc[best_index, "y_list"])
        best_score.append(scores_df.loc[best_index, "match_scores"])
        best_address.append(scores_df.loc[best_index, "clean_address"])
                    
    return_df = pd.DataFrame()
    return_df["best_x"] = best_x
    return_df["best_y"] = best_y
    return_df["best_score"] = best_score
    return_df["best_address"] = best_address
    return return_df

In [326]:
michael_coords["matched_addresses"][48]

'I-10, Florida, USA,I-10 & left lane, Florida, USA,I-10 & i-10 w estate, Florida, USA,I-10 & we street, Florida, USA,I-10 & escambia bay bridge, Florida, USA,I-10 & le fort, Florida, USA,I-10 & blocked la street, Florida, USA,I-10 & i-10 we street, Florida, USA,I-10 & brg left lane, Florida, USA,I-10 & la street, Florida, USA,I-10 & bay bridge, Florida, USA,I-10 & brg le fort, Florida, USA,I-10 & w estate, Florida, USA'

In [327]:
string = "& i-10 we street, Florida"
string.find("i-10")

2

In [352]:
best_location(michael_coords)

,best_x,best_y,best_score,best_address
0,-82.25938808381903,26.894883532315905,87.66,",I-10, mile marker 185, Florida"
1,-82.25987841618097,26.894937456575605,87.66,",I-10, mile marker 182, Florida"
2,-82.26062991618096,26.894936482110833,87.66,",I-10, mile marker 176, Florida"
3,-82.2633615,26.894892532315897,87.66,",I-10, mile marker 153, Florida"
4,-84.867291,29.743254000000007,95.68,",SR 30 & 4 right lane, Florida"
5,-84.8696985,29.741787000000002,95.68,",SR 30 & 2 right lane, Florida"
6,-84.8685735,29.742525000000004,95.68,",SR 30 & 3 right lane, Florida"
7,-82.26292861852951,26.898851607565252,87.24,",I-10, mile marker 95, Florida"
8,-84.69513999999998,30.577000000000055,95.54,",I-10, exit 174, Florida"
9,-85.60409849999999,30.131082,98.63,",SR 30 & pk way, Florida"
